In [1]:
import os

# Hide warnings
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
# Limiting GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

import torch
torch.cuda.empty_cache()   # Emptying Cuda cache in order to free important space
torch.cuda.is_available()

2024-04-29 13:26:51.345256: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 13:26:52.463962: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-29 13:26:54.461691: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-29 13:26:54.462559: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Fo

True

In [2]:
from keras.models import load_model

# Path to the saved model file
model_path = "./model/gemma/gemma_2b_ft.keras"

# Load the model
loaded_model = load_model(model_path)

2024-04-29 13:26:57.131384: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2097152000 exceeds 10% of free system memory.
2024-04-29 13:26:58.168946: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2097152000 exceeds 10% of free system memory.
2024-04-29 13:26:58.399839: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2097152000 exceeds 10% of free system memory.
/home/devy/miniconda3/envs/AI/lib/python3.9/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
2024-04-29 13:35:06.326867: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] All

In [5]:
template = "Query:\n{instruction}\n\nResponse:\n{response}"
prompt=template.format(
instruction='Who are you?',
response='',
)
print(loaded_model.generate(prompt, max_length=256))

I0000 00:00:1714393224.901298     493 service.cc:145] XLA service 0x2b90cd10 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714393224.902104     493 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-04-29 15:20:25.500677: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1714393235.711272     493 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Query:
Who are you?

Response:
I am a large language model, trained by Google. I am a conversational AI that can understand and respond to your questions and requests. I am also able to generate creative text formats, such as poems, scripts, and emails.


In [7]:
import tkinter as tk
from tkinter import scrolledtext

class ChatWindow:
    def __init__(self, master):
        self.master = master
        master.title("Chat Window")

        self.chat_history = scrolledtext.ScrolledText(master, width=60, height=20)
        self.chat_history.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

        self.input_label = tk.Label(master, text="Your message:")
        self.input_label.grid(row=1, column=0, padx=10, pady=5, sticky="w")

        self.input_entry = tk.Entry(master, width=40)
        self.input_entry.grid(row=1, column=1, padx=10, pady=5)

        self.send_button = tk.Button(master, text="Send", command=self.send_message)
        self.send_button.grid(row=2, column=0, columnspan=2, padx=10, pady=5)

    def send_message(self):
        message = self.input_entry.get()
        if message:
            # Display user message in chat history
            self.display_message("You", message)
            # Process user input
            prompt = message + "\n\n"
            response = loaded_model.generate(prompt, max_length=256)
            # Display model response in chat history
            self.display_message("Model", response)
            # Clear input entry
            self.input_entry.delete(0, tk.END)

    def display_message(self, sender, message):
        self.chat_history.configure(state='normal')
        self.chat_history.insert(tk.END, f"{sender}: {message}\n")
        self.chat_history.configure(state='disabled')
        # Scroll to the bottom of the chat history
        self.chat_history.see(tk.END)

def main():
    root = tk.Tk()
    app = ChatWindow(root)
    root.mainloop()

if __name__ == "__main__":
    main()
